# 패키지

In [1]:
import os

In [2]:
import tensorflow as tf     # 텐서플로
import pandas as pd         # 판다스

# GPU 사용 설정

In [3]:
# 사용 가능한 모든 GPU 리스트
gpus = tf.config.list_physical_devices('GPU')

# 사용할 프로세서 선택 : GPU
if gpus:
    try: tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e: print(e)

# 데이터셋

In [4]:
# 데이터셋 주소
dataset_dir = os.path.join('.', 'datasets', 'MusicalSymbol-v.1.1.5')

# CSV 파일 주소
csv_dir = os.path.join(dataset_dir, 'label.csv')

In [9]:
# csv 파일 불러오기
df = pd.read_csv(csv_dir)

# name을 정수값으로 추출
df['name_int'] = df['name'].str.extract('(\d+)').astype(int)

# 정수값을 기준으로 정렬 및 인덱스 초기화
df = df.sort_values('name_int')
df.reset_index(drop=True, inplace=True)

# 필요 없는 name, name_int 열 제거
df = df.drop(columns=['name', 'name_int'])

##########
df = df.filter(like='staff')
df = df.drop(columns=[' staff', ' staff-half-left', ' staff-half-right'])
#
print(df.columns)
##########

# pandas 데이터 프레임을 tensorflow 텐서로 변환
df = tf.convert_to_tensor(df.values, dtype=tf.int32)

Index([' staff-a4-0-0', ' staff-b4-0-0', ' staff-c5-0-0', ' staff-d4-0-0',
       ' staff-d5-0-0', ' staff-e4-0-0', ' staff-e5-0-0', ' staff-f4-0-0',
       ' staff-f5-0-0', ' staff-g4-0-0', ' staff-g5-0-0'],
      dtype='object')


In [10]:
def make_data(img_path):
    # 타겟 생성
    image = tf.io.read_file(img_path)               # 파일 로드
    image = tf.image.decode_png(image, channels=1)  # png 파일로 변환
    image = tf.cast(image, tf.float32)              # uint8 -> float32
    image = image / 255.0                           # 0~1 로 정규화
    image = 1.0 - image                             # 흑백 이미지 반전

    # 레이블 생성
    lable_index = tf.strings.split(img_path, os.path.sep)[-1]           # 파일 이름 추출
    lable_index = tf.strings.split(lable_index, '.')[0]                 # 확장자 제거
    lable_index = tf.strings.to_number(lable_index, out_type=tf.int32)  # 숫자 변환
    lable = tf.gather(df, lable_index-1)                                # df 에서 lable_index 번째 label 추출

    # 타겟과 레이블 반환
    return (image, lable)

def filter_dataset(image, label):
    # label이 모두 0인 경우 필터링
    return tf.math.reduce_any(label != 0)

In [11]:
# 데이터셋 생성
dataset = tf.data.Dataset.list_files(os.path.join(dataset_dir, '*.png'))

# map
dataset = dataset.map(make_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# filter
dataset = dataset.filter(filter_dataset)

# repeat
dataset = dataset.repeat(3)

# shuffle
dataset = dataset.shuffle(buffer_size=25000)

# train, validation
ds_validation = dataset.take(320).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_train = dataset.skip(320).batch(32)

# 모델 학습

In [12]:
# 그래프 초기화
tf.keras.backend.clear_session()

# 모델 생성 및 학습
with tf.device('/device:GPU:0'):
    # 모델 생성
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(192, 512, 1)))
    model.add(tf.keras.layers.MaxPool2D(2, 2))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(2, 2))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(2, 2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(11, activation='softmax'))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # 모델 학습
    model.fit(ds_train, epochs=10, validation_data=ds_validation)

Epoch 1/10
2106/2106 [==============================] - 175s 75ms/step - loss: 0.0035 - accuracy: 0.9990 - val_loss: 6.3330e-08 - val_accuracy: 1.0000
Epoch 2/10
2106/2106 [==============================] - 182s 81ms/step - loss: 3.7960e-08 - accuracy: 1.0000 - val_loss: 3.7625e-08 - val_accuracy: 1.0000
Epoch 3/10
2106/2106 [==============================] - 187s 83ms/step - loss: 9.6412e-09 - accuracy: 1.0000 - val_loss: 1.0803e-08 - val_accuracy: 1.0000
Epoch 4/10
2106/2106 [==============================] - 194s 86ms/step - loss: 3.3983e-09 - accuracy: 1.0000 - val_loss: 2.9802e-09 - val_accuracy: 1.0000
Epoch 5/10
2106/2106 [==============================] - 201s 89ms/step - loss: 1.4152e-09 - accuracy: 1.0000 - val_loss: 3.7253e-10 - val_accuracy: 1.0000
Epoch 6/10
2106/2106 [==============================] - 209s 92ms/step - loss: 8.7036e-10 - accuracy: 1.0000 - val_loss: 1.1176e-09 - val_accuracy: 1.0000
Epoch 7/10
2106/2106 [==============================] - 215s 94ms/step - l